In [1]:
print("hello world")

hello world


#### Pandas external library loading

In [2]:
import pandas as pd

#### Reading data from Excel

In [3]:
# Stock data loading
file_path = "C:\\Python repositories\\Education project\\src\\data\\input\\Stock.xlsx"

stock = pd.read_excel(file_path)
stock.tail(3)

,SKU,Date,Stock
20071,c7d88a1d-1ed1-4aad-84ed-186afde4e4be,2024-06-01,0.7
20072,e47462a3-427a-4727-903a-46fa27578068,2024-06-01,0.7
20073,06bbb978-eefb-4d23-aede-1eb453a21177,2024-06-01,0.7


In [4]:
# COGS data loading
file_path = "C:\\Python repositories\\Education project\\src\\data\\input\\COGS.xlsx"

cogs = pd.read_excel(file_path)
cogs.tail()

,SKU,COGS
5169,25e80579-52af-4e05-9c12-c91c9a02ff29,0.0
5170,bf1b3c3c-155a-4169-b6bc-59e318c10775,0.0
5171,7f41b2f6-b6ee-4808-a2d3-3d96531b51d8,0.0
5172,4823ff97-a12a-48b9-973b-b7ef72dd026e,0.0
5173,e3e70a4d-cc96-439e-9d03-973f73f49957,0.0


In [5]:
# Create a string Period column from Date
# New column
stock['Period'] = stock['Date']
# Change format
stock['Period'] = pd.to_datetime(stock['Date']).dt.strftime('%Y-%m')
# Delete old column
del stock['Date']
stock.tail()

,SKU,Stock,Period
20069,ef8df9d9-3740-45fb-bb23-3ab5a4cf4103,0.7,2024-06
20070,a2d70046-4084-439a-a314-edb49007151d,0.7,2024-06
20071,c7d88a1d-1ed1-4aad-84ed-186afde4e4be,0.7,2024-06
20072,e47462a3-427a-4727-903a-46fa27578068,0.7,2024-06
20073,06bbb978-eefb-4d23-aede-1eb453a21177,0.7,2024-06


In [6]:
cogs.tail()

,SKU,COGS
5169,25e80579-52af-4e05-9c12-c91c9a02ff29,0.0
5170,bf1b3c3c-155a-4169-b6bc-59e318c10775,0.0
5171,7f41b2f6-b6ee-4808-a2d3-3d96531b51d8,0.0
5172,4823ff97-a12a-48b9-973b-b7ef72dd026e,0.0
5173,e3e70a4d-cc96-439e-9d03-973f73f49957,0.0


In [7]:
# Vlookup COGS column to Stock table
stock_cogs = pd.merge(stock, cogs, how='left', on='SKU')
stock_cogs.tail()

,SKU,Stock,Period,COGS
20069,ef8df9d9-3740-45fb-bb23-3ab5a4cf4103,0.7,2024-06,172.676
20070,a2d70046-4084-439a-a314-edb49007151d,0.7,2024-06,197.540
20071,c7d88a1d-1ed1-4aad-84ed-186afde4e4be,0.7,2024-06,93.240
20072,e47462a3-427a-4727-903a-46fa27578068,0.7,2024-06,152.600
20073,06bbb978-eefb-4d23-aede-1eb453a21177,0.7,2024-06,259.000


In [8]:
# New Stock Value column creation
stock_cogs['Value'] = stock_cogs['Stock']*stock_cogs['COGS']
stock_cogs.tail()

,SKU,Stock,Period,COGS,Value
20069,ef8df9d9-3740-45fb-bb23-3ab5a4cf4103,0.7,2024-06,172.676,120.8732
20070,a2d70046-4084-439a-a314-edb49007151d,0.7,2024-06,197.540,138.2780
20071,c7d88a1d-1ed1-4aad-84ed-186afde4e4be,0.7,2024-06,93.240,65.2680
20072,e47462a3-427a-4727-903a-46fa27578068,0.7,2024-06,152.600,106.8200
20073,06bbb978-eefb-4d23-aede-1eb453a21177,0.7,2024-06,259.000,181.3000


In [9]:
# Choose columns in a specific order as copy
required_cols = ['SKU', 'Period', 'Value']
abc_input = stock_cogs[required_cols].copy()
abc_input.tail()

,SKU,Period,Value
20069,ef8df9d9-3740-45fb-bb23-3ab5a4cf4103,2024-06,120.8732
20070,a2d70046-4084-439a-a314-edb49007151d,2024-06,138.2780
20071,c7d88a1d-1ed1-4aad-84ed-186afde4e4be,2024-06,65.2680
20072,e47462a3-427a-4727-903a-46fa27578068,2024-06,106.8200
20073,06bbb978-eefb-4d23-aede-1eb453a21177,2024-06,181.3000


In [11]:
import pandas as pd
import numpy as np

# ABC Classification function
def assign_abc_groups(df: pd.DataFrame, a_threshold: float = 0.80, b_threshold: float = 0.95) -> pd.DataFrame:
    """
    Assigns ABC classification groups to SKUs per Period based on cumulative value contribution.

    Enhancements:
    -------------
    - Handles cases where total period value is 0 by assigning all SKUs in that period to group 'C'.

    Parameters
    ----------
    df : pd.DataFrame
        Input DataFrame with columns ['SKU', 'Period', 'Value'].
    a_threshold : float, optional
        Cumulative contribution cutoff for class A (default=0.80).
    b_threshold : float, optional
        Cumulative contribution cutoff for class B (default=0.95).

    Returns
    -------
    pd.DataFrame
        DataFrame with columns ['SKU', 'Period', 'Value', 'ABC_Group'].

    Raises
    ------
    ValueError
        If required columns are missing.
    TypeError
        If 'Value' is not numeric.
    """

    # === 1. Input validation ===
    required_columns = {'SKU', 'Period', 'Value'}
    if not required_columns.issubset(df.columns):
        missing = required_columns - set(df.columns)
        raise ValueError(f"Missing required columns: {missing}")

    if not pd.api.types.is_numeric_dtype(df['Value']):
        raise TypeError("'Value' column must be numeric (float or int).")

    # Fill missing values in Value column
    df = df.copy()
    df['Value'] = df['Value'].fillna(0.0)

    # === 2. Aggregate total values per SKU/Period ===
    agg_df = (
        df.groupby(['Period', 'SKU'], as_index=False)['Value']
        .sum()
    )

    # === 3. Sort, rank and compute cumulative metrics ===
    agg_df = agg_df.sort_values(['Period', 'Value'], ascending=[True, False])

    # Compute total value per Period
    agg_df['Total_Period_Value'] = agg_df.groupby('Period')['Value'].transform('sum')

    # Compute cumulative value per Period
    agg_df['Cumulative_Value'] = agg_df.groupby('Period')['Value'].cumsum()

    # Handle division safely: if total = 0, set cumulative percent = 1 (forcing C group)
    agg_df['Cumulative_Percent'] = np.where(
        agg_df['Total_Period_Value'] == 0,
        1.0,
        agg_df['Cumulative_Value'] / agg_df['Total_Period_Value']
    )

    # === 4. ABC Group assignment ===
    def classify_abc(cum_pct: float, total_val: float) -> str:
        """
        Determine ABC class:
        - If total value = 0  → assign 'C'
        - Else apply thresholds normally.
        """
        if total_val == 0:
            return 'C'
        if cum_pct <= a_threshold:
            return 'A'
        elif cum_pct <= b_threshold:
            return 'B'
        else:
            return 'C'

    # Apply classification row-wise
    agg_df['ABC_Group'] = agg_df.apply(
        lambda x: classify_abc(x['Cumulative_Percent'], x['Total_Period_Value']),
        axis=1
    )

    # === 5. Merge results back into original DataFrame ===
    result_df = pd.merge(
        df,
        agg_df[['Period', 'SKU', 'ABC_Group']],
        on=['Period', 'SKU'],
        how='left'
    )

    # === 6. Validation ===
    if result_df['ABC_Group'].isna().any():
        raise RuntimeError("ABC group assignment failed for some rows.")

    return result_df

In [12]:
# ABC classification operation
df_result = assign_abc_groups(abc_input)
df_result.tail()

,SKU,Period,Value,ABC_Group
20069,ffb5d13a-0042-471a-98cc-43de9aa6a1be,2024-02,207282.577919,B
20070,ffb5d13a-0042-471a-98cc-43de9aa6a1be,2024-03,207282.577919,B
20071,ffb5d13a-0042-471a-98cc-43de9aa6a1be,2024-04,114196.555298,B
20072,ffb5d13a-0042-471a-98cc-43de9aa6a1be,2024-05,36526.801911,C
20073,ffb5d13a-0042-471a-98cc-43de9aa6a1be,2024-06,188202.999203,B


In [13]:
import os

# Saving in the same folder as the Python (py or ipynb) file using one of two popular Excel libraries
def save_local_file(data: pd.DataFrame, name: str) -> None:
    """
    Saves a DataFrame to an Excel file using the preferred engine.
    Falls back to openpyxl if xlsxwriter is unavailable.
    """
    dump_file_name = f"{name}.xlsx"
    data_dump = os.path.join(os.getcwd(), dump_file_name)

    try:
        # Try xlsxwriter first (faster, supports formatting)
        writer = pd.ExcelWriter(data_dump, engine="xlsxwriter")
    except ModuleNotFoundError:
        print("⚠️  xlsxwriter not found. Falling back to openpyxl.")
        writer = pd.ExcelWriter(data_dump, engine="openpyxl")

    # Write to Excel
    data.to_excel(writer, sheet_name=name, index=False)

    # Save and close the writer properly
    writer.close()
    print(f"✅ Data successfully saved as: {data_dump}")

In [14]:
# Save output file
save_local_file(df_result, "abc")

✅ Data successfully saved as: c:\Python repositories\Education project\src\code\abc.xlsx
